## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [ ]:
%load_ext autoreload
%autoreload 2

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table
from sqlmodel import select

db_url = get_db_url()
print(db_url)
engine = get_engine(db_url)


### optional: create new temp database to work with

In [ ]:
init_db(engine)
s = Session(engine)
objects = [
    StationType(station_type= 'DAVIS' ),
    StationType(station_type= 'SPECTRUM' ), 
    StationType(station_type= 'ONSET'),
    StationType(station_type= 'RAINWISE'),
    StationType(station_type= 'ZENTRA'),
]
s.bulk_save_objects(objects)
s.commit()
station_file = '../data/test_stations.tsv'
# import, using the global 'engine' defined in database module
import_station_file(station_file)


**check that there are stations**

In [ ]:
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()
len(stations)

## Test APIs

 - Davis

In [ ]:

# from ewxpwsdb.weather_apis.weather_api import WeatherAPI, WeatherAPIConfig
from ewxpwsdb.weather_apis import API_CLASS_TYPES


In [ ]:
# get a station of the type we are testing station
def get_one_station(station_type):

    StationAPIClass = API_CLASS_TYPES[station_type]
    
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()
        
    weather_api = StationAPIClass(weather_station)
    
    return(weather_api)

In [ ]:
wapi = get_one_station('SPECTRUM')

In [ ]:
print(wapi.station_type)
print(wapi.sampling_interval)
print(wapi.APIConfigClass)

In [ ]:
# check that configuration class is instantiated with same data in database
api_config = wapi.APIConfigClass.model_validate_json_str(wapi.weather_station.api_config)
print(api_config == wapi.api_config)

In [ ]:
r = wapi.get_readings()


In [ ]:
r[0].response_text

In [ ]:

readings = wapi.transform(r)
readings

## Clean up

remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(StationType))
        session.exec(delete(WeatherStation))
        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()